In [36]:
!pip install pydicom
import pydicom
import glob
import tifffile as tiff
import numpy as np
import cv2 as cv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
def create_DICOM(image_array, Series_Description, output_path):
    # Create a new DICOM dataset
    ds = pydicom.Dataset()

    # Set DICOM tags for required fields
    ds.SeriesDescription = Series_Description
    ds.Rows = image_array.shape[0]
    ds.Columns = image_array.shape[1]
    ds.BitsAllocated = 8
    ds.BitsStored = 8
    ds.HighBit = ds.BitsStored - 1
    ds.PixelRepresentation = 0  # Unsigned integer
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"  # Inverse grayscale

    # Convert the NumPy array to a bytes object
    image_bytes = image_array.tobytes()

    # Set the pixel data in the DICOM dataset
    ds.PixelData = image_bytes

    # Set the appropriate endianness and VR encoding
    ds.is_little_endian = True
    ds.is_implicit_VR = True

    # Create the FMI dataset
    fmi = pydicom.dataset.FileMetaDataset()
    fmi.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.128.1'  # PET/CT Image Storage
    fmi.MediaStorageSOPInstanceUID = '1.2.3.4.5.6.7.8.9.0'  # Generate a unique UID for your instance
    fmi.TransferSyntaxUID = '1.2.840.10008.1.2.1'  # Explicit VR Little Endian

    # Set the FMI dataset in the main dataset
    ds.file_meta = fmi

    # Save the DICOM dataset to a file
    filename = Series_Description + '.dcm'
    ds.save_as(output_path + filename)

In [38]:
path_list = glob.glob('/content/drive/MyDrive/Projetos Python/Image Processing/DATA SPECTRUM DIGITAL PHANTOM/JSNM mask/*.tif')
for path in path_list:
  name = path.split('/')[-1].split('.')[0]
  image = tiff.imread(path)
  channels = cv.split(image)
  image = cv.merge([channels[0]]) 
  create_DICOM(image, name, '/content/drive/MyDrive/Projetos Python/Image Processing/Hoffman Templates/')